<a href="https://colab.research.google.com/github/Prompt-Sopa/CODERHOUSE-NLP-DL/blob/main/TP_Coder_Alsop_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Trabajo Final - CODERHOUSE

- Alumno: Agustin Alsop

##Actividad 2

Deberás entregar un dataset del estilo texto (libro, paper, documento, colección de tweets, etc) donde se desarrollen de mínima dos de las tareas más usuales de preprocesamiento de NLP: steeming, tokenización, remoción de stopwords, lematización, etc.

##Resolución

El corpus elegido es un pdf de un diccionario de lunfardo argentino, la idea es leer el pdf realizar procesamiento de texto a las definiciones de las palabras para convertirlas en vectores y luego comparar con una definicion puesta por el usuario

In [1]:
!pip install pdfplumber
!python -m spacy download es_core_news_sm
!python -m spacy validate
!gdown 1HUJ2RR---Le8l11ExWOtOCYABHHRg0G4
!pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 32.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.8.4) =================
ℹ spaCy installation: /usr/local/lib/python3.11/dist-packages/spacy

NAME        

In [2]:
import pdfplumber
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import es_core_news_sm
import unicodedata
from sentence_transformers import SentenceTransformer, util
from prettytable import PrettyTable

In [3]:
modelo = SentenceTransformer('distiluse-base-multilingual-cased-v1')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
stop_words = set(stopwords.words('spanish'))
nlp = es_core_news_sm.load()

def remove_specials(text):
    return re.sub(r'[^\w\s]', '', text)

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

def remove_stopwords(text):
    """Función para eliminar palabras comunes en español."""
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)

def lematizar_texto(texto):
    """Función para lematizar un texto en español."""
    doc = nlp(texto)
    lemmas = [tok.lemma_.lower() for tok in doc]
    return ' '.join(lemmas)

def procesar_texto(texto):
    """Función para procesar un texto."""
    texto = texto.lower()
    texto = remove_specials(texto)
    texto = remove_stopwords(texto)
    texto = lematizar_texto(texto)
    texto = remove_accents(texto)
    return texto

with pdfplumber.open('/content/Lunfardo.pdf') as pdf:
    pages = pdf.pages[:-2]
    text = ''
    for page in pages:
        text += page.extract_text()

# Luego procesar el texto como antes
lines = text.split('\n')

# Crear una lista para almacenar los datos
data = []

# Procesar cada línea
for line in lines:
    if ':' in line:  # Asegurarse de que la línea tiene el formato correcto
        key, value = line.split(':', 1)  # Separar clave y valor
        definicion = value
        tranformacion = procesar_texto(value)
        definicion_tokens = word_tokenize(tranformacion)
        embeddings = modelo.encode(definicion, convert_to_tensor=True)
        data.append({'Palabra': key.strip(), 'Definición': definicion,'Tranformación': tranformacion, 'Token': definicion_tokens, 'Embeddings': embeddings})

# Convertir la lista en un DataFrame
df = pd.DataFrame(data)
df

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.24k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


,Palabra,Definición,Tranformación,Token,Embeddings
0,Abanico,"Agente policial; delator, soplón; puerta.",agente policial delator soplon puerta,"[agente, policial, delator, soplon, puerta]","[tensor(0.0468, device='cuda:0'), tensor(-0.00..."
1,Abarajar,Parar con el cuchillo los golpes del adversar...,parar cuchillo golpe adversario agarrar asir,"[parar, cuchillo, golpe, adversario, agarrar, ...","[tensor(-0.0089, device='cuda:0'), tensor(-0.0..."
2,Abatatar(se),"Turbarse, perder la serenidad y el libre uso ...",turbar el perder serenidad libre uso facultad,"[turbar, el, perder, serenidad, libre, uso, fa...","[tensor(0.0046, device='cuda:0'), tensor(-0.03..."
3,Abisagrar(se),Casarse.,casar el,"[casar, el]","[tensor(0.0135, device='cuda:0'), tensor(0.002..."
4,Abocar(se),Lanzarse precipitadamente.,lanzar el precipitadamente,"[lanzar, el, precipitadamente]","[tensor(0.0064, device='cuda:0'), tensor(-0.05..."
...,...,...,...,...,...
1425,Zingar,Copular.,copular,[copular],"[tensor(0.0367, device='cuda:0'), tensor(-0.04..."
1426,Zíngaro,Gitano.,gitano,[gitano],"[tensor(0.0668, device='cuda:0'), tensor(0.000..."
1427,Zoquete,Pedazo de carne ordinaria y dura; bofetada; s...,pedazo carne ordinario duro bofetada aplico pe...,"[pedazo, carne, ordinario, duro, bofetada, apl...","[tensor(-0.0209, device='cuda:0'), tensor(-0.0..."
1428,Zurda,La izquierda política.,izquierda politico,"[izquierda, politico]","[tensor(0.0135, device='cuda:0'), tensor(-0.00..."


In [4]:
input_str = input("Ingrese una frase: ")
input_str = procesar_texto(input_str)

input_tokens = set(word_tokenize(input_str))

df['coincidencia_frec'] = df['Token'].apply(lambda x: len(set(x) & input_tokens))

df = df.sort_values(by='coincidencia_frec', ascending=False)
df[['Palabra', 'Token', 'coincidencia_frec']]

#persona que marcha por politica principalmente por el movimiento de izquierda

Ingrese una frase: persona que marcha por politica principalmente por el movimiento de izquierda


,Palabra,Token,coincidencia_frec
690,Lechuza,"[persona, atribuir, influjo, malefico, princip...",2
1429,Zurdo,"[militar, izquierdo, politico, profes, ideologia]",2
97,Bachicha,"[persona, italiano, principalmente, si, genove...",2
883,Oligarca,"[persona, perteneciente, clase, social, ejerce...",1
247,Camelo,"[engano, faltar, verdad, principalmente, produ...",1
...,...,...,...
485,Esquinazo,"[acto, plantar, dejar, el, burlado, abandonado]",0
484,Esquiyar,"[impacientar, el, rabiar]",0
483,Esquifuso,"[asqueroso, repugnante]",0
482,Esquifrunista,"[ladron, complicidad, prostituto, despojo]",0


In [5]:
nlp = es_core_news_sm.load()

input_str = input("Ingrese una frase: ")

df['coincidencia_spacy'] = df['Tranformación'].apply(lambda x: nlp(x).similarity(nlp(input_str)))

df = df.sort_values(by='coincidencia_spacy', ascending=False)

df[['Palabra', 'Definición', 'coincidencia_spacy']]

#persona que marcha por politica principalmente por el movimiento de izquierda

Ingrese una frase: persona que marcha por politica principalmente por el movimiento de izquierda


<ipython-input-5-0248beaccead>:5: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  df['coincidencia_spacy'] = df['Tranformación'].apply(lambda x: nlp(x).similarity(nlp(input_str)))
<ipython-input-5-0248beaccead>:5: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  df['coincidencia_spacy'] = df['Tranformación'].apply(lambda x: nlp(x).similarity(nlp(input_str)))


,Palabra,Definición,coincidencia_spacy
0,Abanico,"Agente policial; delator, soplón; puerta.",0.615946
545,Fondín,Casa de comida de baja condición.,0.611034
389,Crosta,Individuo o cosa de baja condición.,0.576391
1109,Rascabuche,Persona que come malamente; persona de baja c...,0.564750
243,Calo,"Pedazo, parte de una cosa separada de otra, s...",0.563688
...,...,...,...
66,Apompar (se),Aturdirse.,-0.107771
1319,Trovar,Encontrar.,-0.112103
1034,Pishar,Orinar.,-0.140167
1027,Pipí,Orina.,-0.140167


In [6]:
input_str = input("Ingrese una frase: ")

# Codificar la entrada
input_embeddings = modelo.encode(input_str, convert_to_tensor=True)

# Calcular la similitud coseno entre la entrada y cada definición
df['coincidencia_sbert'] = df['Embeddings'].apply(lambda x: util.cos_sim(x, input_embeddings).item())

df = df.sort_values(by='coincidencia_sbert', ascending=False)

df[['Palabra', 'Definición', 'coincidencia_sbert']]

#persona que marcha por politica principalmente por el movimiento de izquierda

Ingrese una frase: persona que marcha por politica principalmente por el movimiento de izquierda


,Palabra,Definición,coincidencia_sbert
1429,Zurdo,El que milita en la izquierda política y el q...,0.627297
1428,Zurda,La izquierda política.,0.521422
883,Oligarca,Persona perteneciente a la clase social que e...,0.411781
690,Lechuza,Persona a la que se atribuye influjo maléfico...,0.410430
1348,Veintiuna,Posición que toman las personas cuando inclin...,0.370876
...,...,...,...
866,Nene,Sujeto de temer.,-0.091069
1147,Requecho,"Desecho, sobra.",-0.099374
369,Colimbo,Recluta.Comedor: Dentadura.,-0.105136
1238,Tallove,Botella.,-0.110436
